In [1]:
## On Not being able to use python for Despina's. She said in L10 that the only thing she wants to see is that we've done it and that it wont be marked.
## Only assessed on whether it works or not

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import norm
from datetime import datetime, time 

## Importing hand-made modules
import sys
scraper_modules_path = 'C:/Users/ABour/Desktop/Energy Systems & Data Analytics/Modules/Energy Analysis - Aidan/Smart Meters/custom_modules'
if scraper_modules_path not in sys.path:
    sys.path.append(scraper_modules_path)

import pre_processing as pp
import feature_gen as fg

<br>

### Reading in and time series feature generation

In [2]:
batch = 5
df_smart_meters = pd.read_pickle(f'data/batch_{batch}.pkl')

In [3]:
df_smart_meters = df_smart_meters.drop(columns='stdorToU').copy()

std_bool = df_smart_meters.datetime.dt.year!=2013
df_smart_meters_std = df_smart_meters.copy()[std_bool]
df_smart_meters_ToU = df_smart_meters.copy()[~std_bool]

df_smart_meters_ToU

,id,datetime,kWh
134158912,MAC000005,2013-01-01 00:00:00,0.096
134158913,MAC000005,2013-01-01 00:30:00,0.111
134158914,MAC000005,2013-01-01 01:00:00,0.071
134158915,MAC000005,2013-01-01 01:30:00,0.035
134158916,MAC000005,2013-01-01 02:00:00,0.035
134158917,MAC000005,2013-01-01 02:30:00,0.034
134158918,MAC000005,2013-01-01 03:00:00,0.035
134158919,MAC000005,2013-01-01 03:30:00,0.03
134158920,MAC000005,2013-01-01 04:00:00,0.028
134158921,MAC000005,2013-01-01 04:30:00,0.03


In [4]:
print(f'ToU Size: {df_smart_meters_ToU.shape[0]}')
print(f'Std Size: {df_smart_meters_std.shape[0]}')

ToU Size: 18892362
Std Size: 14891409


In [5]:
df_smart_meters_ToU = df_smart_meters_ToU[df_smart_meters_ToU.datetime!=datetime(2013, 1, 1, 0, 0)] # Drop 1st HH of 2013
df_smart_meters_ToU

,id,datetime,kWh
134158913,MAC000005,2013-01-01 00:30:00,0.111
134158914,MAC000005,2013-01-01 01:00:00,0.071
134158915,MAC000005,2013-01-01 01:30:00,0.035
134158916,MAC000005,2013-01-01 02:00:00,0.035
134158917,MAC000005,2013-01-01 02:30:00,0.034
134158918,MAC000005,2013-01-01 03:00:00,0.035
134158919,MAC000005,2013-01-01 03:30:00,0.03
134158920,MAC000005,2013-01-01 04:00:00,0.028
134158921,MAC000005,2013-01-01 04:30:00,0.03
134158922,MAC000005,2013-01-01 05:00:00,0.033


In [6]:
## Reading in ToU price data
df_ToU_price_raw = pd.read_csv('data/tariff_d.csv', parse_dates=['GMT']).rename({'GMT':'datetime', 'Price':'price', 'Event_tags':'event'}, axis=1)

## Cleaning and extracting price 
def clean_and_extract_price(df_ToU_price):
    df_ToU_price = df_ToU_price[df_ToU_price.datetime.dt.year==2013] # Dropping spare day at start of 2014
    df_ToU_price = df_ToU_price.drop(columns=['event']).set_index('datetime') # Drop unnecessary columns, set datetime as index
    return df_ToU_price

df_price = clean_and_extract_price(df_ToU_price_raw.copy())
df_price = pd.Series(df_price.price)
df_price

datetime
2013-01-01 00:30:00    0.1176
2013-01-01 01:00:00    0.1176
2013-01-01 01:30:00    0.1176
2013-01-01 02:00:00    0.1176
2013-01-01 02:30:00    0.1176
2013-01-01 03:00:00    0.1176
2013-01-01 03:30:00    0.1176
2013-01-01 04:00:00    0.1176
2013-01-01 04:30:00    0.1176
2013-01-01 05:00:00    0.1176
2013-01-01 05:30:00    0.1176
2013-01-01 06:00:00    0.1176
2013-01-01 06:30:00    0.1176
2013-01-01 07:00:00    0.1176
2013-01-01 07:30:00    0.1176
2013-01-01 08:00:00    0.1176
2013-01-01 08:30:00    0.1176
2013-01-01 09:00:00    0.1176
2013-01-01 09:30:00    0.1176
2013-01-01 10:00:00    0.1176
2013-01-01 10:30:00    0.1176
2013-01-01 11:00:00    0.1176
2013-01-01 11:30:00    0.1176
2013-01-01 12:00:00    0.1176
2013-01-01 12:30:00    0.1176
2013-01-01 13:00:00    0.1176
2013-01-01 13:30:00    0.1176
2013-01-01 14:00:00    0.1176
2013-01-01 14:30:00    0.1176
2013-01-01 15:00:00    0.1176
                        ...  
2013-12-31 09:00:00    0.1176
2013-12-31 09:30:00    0.1176
2

In [7]:
print(df_smart_meters_ToU[df_smart_meters_ToU.datetime.dt.second != 0])
print('')
print(df_smart_meters_ToU[~(df_smart_meters_ToU.datetime.dt.minute.isin([0, 30]))])

                  id            datetime   kWh
143691393  MAC001477 2013-10-29 09:33:01  Null
144709892  MAC001653 2013-10-29 09:33:01  Null
153932659  MAC003155 2013-07-05 12:54:31  Null
155953520  MAC003559 2013-10-29 18:23:02  Null
167382338  MAC005510 2013-09-20 18:24:09  Null

                  id            datetime   kWh
143691393  MAC001477 2013-10-29 09:33:01  Null
144709892  MAC001653 2013-10-29 09:33:01  Null
153932659  MAC003155 2013-07-05 12:54:31  Null
155953520  MAC003559 2013-10-29 18:23:02  Null
167382338  MAC005510 2013-09-20 18:24:09  Null


In [8]:
df_smart_meters_ToU.at[143691393, 'datetime'] = datetime(2013, 10, 29, 9, 30)
df_smart_meters_ToU.at[144709892, 'datetime'] = datetime(2013, 10, 29, 9, 30)
df_smart_meters_ToU.at[153932659, 'datetime'] = datetime(2013, 7, 5, 13, 0)
df_smart_meters_ToU.at[155953520, 'datetime'] = datetime(2013, 10, 29, 18, 30)
df_smart_meters_ToU.at[167382338, 'datetime'] = datetime(2013, 9, 20, 18, 30)

df_smart_meters_ToU[~(df_smart_meters_ToU.datetime.dt.minute.isin([0, 30]))].shape[0]

0

In [9]:
## Filtering ToU df for only std prices

def datetime_2_price(datetime):
    price = df_price.loc[datetime]
    return price

df_smart_meters_ToU['price'] = df_smart_meters_ToU.datetime.apply(datetime_2_price)
df_smart_meters_ToU = df_smart_meters_ToU[df_smart_meters_ToU.price==0.1176]
df_smart_meters_ToU = df_smart_meters_ToU.drop(columns='price')
df_smart_meters_ToU

C:\Users\ABour\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,id,datetime,kWh
134158913,MAC000005,2013-01-01 00:30:00,0.111
134158914,MAC000005,2013-01-01 01:00:00,0.071
134158915,MAC000005,2013-01-01 01:30:00,0.035
134158916,MAC000005,2013-01-01 02:00:00,0.035
134158917,MAC000005,2013-01-01 02:30:00,0.034
134158918,MAC000005,2013-01-01 03:00:00,0.035
134158919,MAC000005,2013-01-01 03:30:00,0.03
134158920,MAC000005,2013-01-01 04:00:00,0.028
134158921,MAC000005,2013-01-01 04:30:00,0.03
134158922,MAC000005,2013-01-01 05:00:00,0.033


In [10]:
def pre_proc_ts_feature_gen(df):
    df = pp.generate_ts_features_df(df)
    df['kWh'] = df.kWh.replace('Null', None).astype(float)
    return df

df_smart_meters_std = pre_proc_ts_feature_gen(df_smart_meters_std)
df_smart_meters_ToU = pre_proc_ts_feature_gen(df_smart_meters_ToU)

In [11]:
plot_households = False

if plot_households == True:
    for _id in df_smart_meters.id.unique():
        ## Filter df for smart meter id
        df_smart_meter = df_smart_meters[df_smart_meters.id==_id]

        ## Plots
        pp.plot_SP_mean(_id, df_smart_meter)
        pp.plot_datetime_heatmap(_id, df_smart_meter)
        pp.plot_daily_peak_dist(_id, df_smart_meter)
    
## Try dropping zero values before rather than later, compare speed

<br>

### Feature Generation

<img src="img/SMAP features.png">

In [12]:
## Creating array of SMAP feature keywords

## Need to add a pre-pre-filter for night & weekend combo, five so far
## They have mean daily maximum usage as well as mean peak usage, are they not the same? 
## Need a method to filter out peak usage
## Need to add seasonal filters

SMAP_features = np.array([
    [None, None, None, 'mean'], # Mean usage
    [None, None, None, 'var'], # Variance of usage
    [None, 'DoY', 'max', 'mean'], # Mean peak usage
    [None, 'DoY', 'max', 'var'], # Variance of peak usage
    ['day', None, None, 'mean'], # Mean night usage
    ['night', None, None, 'mean'], # Mean day usage
    ['day', None, None, 'var'], # Variance of night usage
    ['weekend', None, None, 'mean'], # Mean weekday usage
    ['weekend', None, None, 'var'], # Variance of weekday usage
    [None, 'DoY', 'min', 'mean'], # Mean trough usage
    [None, 'SP', 'var', 'mean'], # Mean of half-hourly coefficients of variation
    ['not_lunch', None, None, 'mean'], # Mean lunchtime usage
    [None, None, None, 'max'], # Peak usage
    [None, None, None, 'min'], # Trough usage
    ['night', None, None, 'var'], # Variance of day usage
    ['weekend', 'DoY', 'max', 'mean'], # Mean weekday peak usage
    ['weekend', 'DoY', 'max', 'var'], # Variance of weekday peak usage
    ['weekday', None, None, 'mean'], # Mean weekend usage
    ['weekday', None, None, 'var'], # Variance of weekend usage
    [None, 'SP', 'mean', None], # Mean of half-hourly usage
    [None, 'SP', 'var', None], # Variance of half-hourly usage
])

In [13]:
def generate_household_features(df_smart_meters):
    house_dict = dict()

    for _id in df_smart_meters.id.unique():
        house_dict[_id] = dict() # Creating dictionary entry for new household id
        df_smart_meter = df_smart_meters[df_smart_meters.id==_id] # Filter df for smart meter id

        for i in range(SMAP_features.shape[0]):
            drop, time_step, time_step_feature, final_feature = SMAP_features[i]
            house_dict = fg.feature_gen(df_smart_meter, _id, house_dict, drop=drop, time_step=time_step, time_step_feature=time_step_feature, final_feature=final_feature)

    return house_dict

ToU_household_dict = generate_household_features(df_smart_meters_ToU)
std_household_dict = generate_household_features(df_smart_meters_std)

In [14]:
def household_dict_2_df(house_dict):
    df_households = pd.DataFrame(house_dict).T
    df_households['day/night'] = df_households['drop_night_combined_mean']/df_households['drop_day_combined_mean']

    df_households.index.name = 'household_id'
    return df_households

df_ToU = household_dict_2_df(ToU_household_dict)
df_std = household_dict_2_df(std_household_dict)

In [15]:
df_ToU.to_csv(f'data/ToU_ts_features_batch_{batch}.csv')
df_std.to_csv(f'data/std_ts_features_batch_{batch}.csv')